# [Angle closure Glaucoma Evaluation Challenge](https://age.grand-challenge.org/Details/)
## Angle closure classification Baseline

## Inference

- Assume `Training100.zip` and `Validation_ASOCT_Image.zip` are stored @ `./AGE_challenge Baseline/datasets/`
- Assume `weights` are stored @ `./AGE_challenge Baseline/weights/`
- In training phase, we use standard ResNet34 with `sigmoid(fc(1))` output
- We split a single image into two parts

In [1]:
import os, random, functools, math
import cv2
import numpy as np
import time
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve

In [2]:
import paddle
import paddle.fluid as fluid
import paddle.fluid.layers as FL
import paddle.fluid.optimizer as FO
fluid.install_check.run_check()

Running Verify Fluid Program ... 
Your Paddle Fluid works well on SINGLE GPU or CPU.
Your Paddle Fluid works well on MUTIPLE GPU or CPU.
Your Paddle Fluid is installed successfully! Let's start deep Learning with Paddle Fluid now


In [3]:
from resnet import *

In [4]:
data_root_path = "../datasets/Training100/"
image_path = os.path.join(data_root_path, "ASOCT_Image")

val_file_path = os.path.join(data_root_path, "cls_val_split.csv")

output_file = "./Classification_Results.csv"

In [5]:
BATCH_SIZE = 32 // 2 # image split * 2
THREAD = 8
BUF_SIZE = 32

### Define Inference Data Loader

In [6]:
# Real time data augmentation
def crop_image(img, target_size, center):
    """ crop_image """
    height, width = img.shape[:2]
    size = target_size
    if center == True:
        w_start = (width - size) // 2
        h_start = (height - size) // 2
    else:
        w_start = np.random.randint(0, width - size + 1)
        h_start = np.random.randint(0, height - size + 1)
    w_end = w_start + size
    h_end = h_start + size
    img = img[h_start:h_end, w_start:w_end, :]
    return img

def split_image(img):
    rows,_,_ = img.shape
    # left, right split
    return [img[:, :rows, :], img[:, -rows:, :]]
    
# data reader and xmap wrapper to enable multiprocessing data load

def reader(img_path, file_list, batch_size=32, shuffle=True, shuffle_seed=42):
    def read_file_list():
        batch_data = []
        np.random.shuffle(file_list)
        for line in file_list:
            single_img_path, _, _ = line.split(",")
            batch_data.append(single_img_path)
            if len(batch_data) == batch_size:
                yield batch_data
                batch_data = []
        if len(batch_data) != 0:
            yield batch_data
    return read_file_list

def process_batch_data(input_data):
    batch_data = []
    for sample in input_data:
        file = sample

        img = cv2.imread( file )
        img = img[:, :, ::-1].astype('float32') / 255
        
        img = np.concatenate(split_image(img), axis=-1) # concat at channel dim
        img = cv2.resize(img, (256, 256))
        
        img = crop_image(img, target_size=224, center=True)
        
        img = img.transpose((2, 0, 1))

        batch_data.append((file, 0, img[:3,:,:]))
        batch_data.append((file, 1, img[3:,:,:]))

    return batch_data

In [7]:
def data_loader(img_list, img_path, batch_size, order=False):
    data_reader = reader(img_path, img_list, batch_size)
    mapper = functools.partial(process_batch_data)
    
    data_reader = paddle.reader.shuffle(data_reader, 32)
    
    return paddle.reader.xmap_readers(
        mapper, data_reader, THREAD, BUF_SIZE, order=order)

In [8]:
with open(val_file_path) as flist:
    val_file_list = [os.path.join(image_path,line.strip()) for line in flist]
    
val_dataloader = data_loader(val_file_list, image_path, BATCH_SIZE, True)

### Define model (compute graph)

In [9]:
def network():
    data_shape = [3, 224, 224]
    
    model = ResNet34()
    
    input_feature = FL.data(name='pixel', shape=data_shape, dtype='float32')
    
    logit = model.net(input_feature, class_dim=1)
    predict = FL.sigmoid(logit)

    return predict

In [10]:
def inference(use_cuda, pretrained_model, threshold=0.5):
    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
    
    startup_prog = fluid.Program()
    val_prog = fluid.Program()

    # 定义预测网络
    with fluid.program_guard(val_prog, startup_prog):
        # Use fluid.unique_name.guard() to share parameters with train network
        with fluid.unique_name.guard():
            val_output = network()

    val_prog = val_prog.clone(for_test=True)
    val_output.persistable = True
            
    exe = fluid.Executor(place)
    exe.run(startup_prog)

    if pretrained_model:
        def if_exist(var):
            return os.path.exists(os.path.join(pretrained_model, var.name))

        fluid.io.load_vars(
            exe, pretrained_model, main_program=val_prog, predicate=if_exist)
    
    positive_ratio = 1. / (1. - threshold)
    negative_ratio = 1. / threshold
    
    result = {}
    for tid, data in enumerate(val_dataloader()):
        file_names, part_splits, val_datas = [],[],[]
        for item in data:
            file_names.append(item[0])
            part_splits.append(item[1])
            val_datas.append(item[2])
        
        batch_preds, = exe.run(
           program=val_prog,
           feed={"pixel":np.array(val_datas)},
           fetch_list=[val_output],
           use_program_cache=True)

        for file, part, pred in zip(file_names, part_splits, batch_preds[:,0]):
            if pred >= threshold:
                threshold_pred = (pred-threshold) * positive_ratio
            else:
                threshold_pred = (pred-threshold) * negative_ratio
            if file not in result.keys():
                result[file] = [0, 0]
            result[file][part] = threshold_pred
    return result
        

In [13]:
result = inference(True, "../weights/classify_weights_best/")

In [14]:
with open(output_file, "w+") as f:
    f.write("{},{},{}\n".format("ASOCT_NAME", "LEFT_ANGLE_RESULTS", "RIGHT_ANGLE_RESULTS"))
    for file, pred_labels in result.items():
        f.write("{},{},{}\n".format(file.split("/")[-1], *pred_labels))